In [1]:
from ultralytics import YOLO
import cv2
import cvzone
import math
from Sort import *

In [5]:
cap=cv2.VideoCapture('cars.mp4')

model=YOLO('weights_yolo12n.pt')

classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

mask = cv2.imread('mask.png')

tracker=Sort(max_age=20, min_hits=3, iou_threshold=0.3)

limits= [400, 297, 673, 297]

totalcount=[]

while True:
    success, img=cap.read()
    imgRegion= cv2.bitwise_and(img,mask)
    results=model(imgRegion, stream=True)

    detections=np.empty((0,5))
    
    for r in results:
        #BoundingBoxes
        boxes=r.boxes
        for box in boxes:
            x1, y1, x2, y2=box.xyxy[0]
            x1, y1, x2, y2=int(x1), int(y1), int(x2), int(y2)
            # cv2.rectangle(img, (x1,y1), (x2,y2), (255, 0, 255), 3)
            w, h= x2 - x1, y2 - y1

            #Confidence
            conf=math.ceil((box.conf[0] * 100)) /100
            
            #ClassNames
            cls=int(box.cls[0])

            currentClass = classNames[cls]
            if currentClass in ['car', 'motorbike', 'bus', 'truck'] and conf > 0.3:
                # cvzone.putTextRect(img, f'{currentClass} {conf}', (max(0 ,x1), max(35 ,y1)),
                #                scale=0.8, thickness=1, offset=5)
                
                # cvzone.cornerRect(img, (x1,y1,w,h), l=7)
                currentArray=np.array([x1, y1, x2, y2, conf])
                detections=np.vstack((detections, currentArray))
                
    tracker_result=tracker.update(detections)
    cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0, 0, 255), 5)
   
    for result in tracker_result:
        x1, y1, x2, y2, id = result 
        x1, y1, x2, y2=int(x1), int(y1), int(x2), int(y2)
        w, h= x2 - x1, y2 - y1
        print(result)
        cvzone.cornerRect(img, (x1,y1,w,h), l=7, rt=2, colorR=(255,0,255))
        cvzone.putTextRect(img, f'{currentClass}', (max(0 ,x1), max(35 ,y1)),colorR=(255,0,255),
                               scale=1, thickness=1, offset=6)

        cx, cy= x1 + w //2, y1 + h // 2
        cv2.circle(img, (cx, cy), 2, (255,0,255), cv2.FILLED)
        
        if limits[0]<cx<limits[2] and limits[1] - 30 < cy < limits[1] + 30:
            if totalcount.count(id)==0:
                totalcount.append(id)
                cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0, 255, 0), 5)

                
    cvzone.putTextRect(img, f'Count {len(totalcount)}', (50, 50),colorR=(255,0,255))
                              
    cv2.imshow('image',img)
    cv2.waitKey(1)


0: 384x640 4 cars, 2 trucks, 71.6ms
Speed: 11.2ms preprocess, 71.6ms inference, 7.0ms postprocess per image at shape (1, 3, 384, 640)
[        524         173         558         198         245]
[        571         316         650         428         244]
[        377         348         495         458         243]
[        572         316         651         428         242]
[        377         349         494         456         241]
[        456         226         509         270         240]

0: 384x640 3 cars, 1 truck, 99.7ms
Speed: 8.2ms preprocess, 99.7ms inference, 8.6ms postprocess per image at shape (1, 3, 384, 640)
[     568.99      320.02      650.01      434.98         244]
[     373.87      349.12      492.13      459.88         243]
[     568.16      320.78      648.85      434.21         242]
[     453.86      227.12      507.14      271.88         240]

0: 384x640 5 cars, 2 trucks, 186.6ms
Speed: 10.8ms preprocess, 186.6ms inference, 12.0ms postprocess per image 

KeyboardInterrupt: 